In [143]:
from bs4 import BeautifulSoup
import os
import requests
import json
from selenium import webdriver
import time
import pickle

In [ ]:
base_URL = "https://forum.arbitrum.foundation"
# response = requests.get(base_URL)
# soup = BeautifulSoup(response.content,"html.parser")


In [114]:
# for understanding the structure of elements
# with open("file1.html","w")as ff:
    # ff.write(str(soup))
# with open("file1.html","w")as ff:
#     ff.write(str(soup))


In [115]:
categories_and_links={}
topics_and_links={}
content={}

In [140]:
def scrape_first_page_categories(url):
    driver = webdriver.Chrome()
    driver.get(url)
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(100):
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        current_height = driver.execute_script("return document.body.scrollHeight")
        if(current_height==last_height):
            break 
        last_height=current_height
    

    
    soup = BeautifulSoup(driver.page_source,"html.parser")
    driver.quit()
    article_categories = {
        'category':[],   
        'links':[]       
    }
    table = soup.find("table",class_='category-list')
    for i in table.find_all("h3"):
        article_categories['links'].append(url+i.a.attrs['href'][1:])
        article_categories['category'].append(i.span.text)

    return article_categories



def scrape_second_page_topics(article_categories):
    article_topics = {}
    for i in range(len(article_categories['category'])):
        url = article_categories['links'][i]
        category = article_categories['category'][i]

        driver = webdriver.Chrome()
        driver.get(url)
        last_height = driver.execute_script("return document.body.scrollHeight")
        for i in range(100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            current_height = driver.execute_script("return document.body.scrollHeight")
            if(current_height==last_height):
                break 
            last_height=current_height
    

    
        soup = BeautifulSoup(driver.page_source,"html.parser")
        driver.quit()

        table = soup.find("table",class_="topic-list")
        record = {
            'title':[],   
            'link':[]       
        }
        for i in table.find_all("td",class_="main-link"):
            record['link'].append(i.a.attrs['href'])
            record['title'].append(i.a.text)

        article_topics[category]= record

    return article_topics    


        



    
    
    

In [ ]:
cc = scrape_second_page_topics(scrape_first_page_categories(base_URL))

In [142]:
with open("../file1.json","w") as ff:
    json.dump(cc,ff)

In [147]:
with open("till_second_page.pkl","wb") as filer:
    pickle.dump(cc,filer)